# Machine Learning

In [33]:
import warnings

import numpy as np
import pandas as pd
from keras.metrics import TopKCategoricalAccuracy
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
train = pd.read_csv('./data/train_df.csv', index_col=0)

In [5]:
le = LabelEncoder()
train['target'] = le.fit_transform(train['country_destination'])
lb = LabelBinarizer()
lb.fit(train['target'])

LabelBinarizer()

In [6]:
mapping = dict(zip(range(len(le.classes_)), le.classes_))
mapping

{0: 'AU',
 1: 'CA',
 2: 'DE',
 3: 'ES',
 4: 'FR',
 5: 'GB',
 6: 'IT',
 7: 'NDF',
 8: 'NL',
 9: 'PT',
 10: 'US',
 11: 'other'}

In [7]:
target = train['target']
feature = train.drop(['target', 'country_destination'], axis=1)

In [8]:
print(target.shape, feature.shape)

(213451,) (213451, 125)


### Baseline Model

In [16]:
# X_train, X_test, y_train, y_test = train_test_split(
#     feature, target, train_size=.25, random_state=42)

In [45]:
def base_model():
    model = Sequential()
    model.add(Dense(120, input_dim=125, activation='relu'))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(12, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.TopKCategoricalAccuracy(k=5)])
    return model

In [ ]:
estimator = KerasClassifier(build_fn=base_model, epochs=5, batch_size=2, verbose=2)
kfold = KFold(n_splits=3, shuffle=True)
results = cross_val_score(estimator, feature, lb.transform(target), cv=kfold)

Epoch 1/5
 - 58s - loss: 1.0838 - accuracy: 0.6314
Epoch 2/5


In [41]:
print("Baseline: %.2f%%" % (results.mean()*100))

Baseline: 94.12%


In [54]:
fit_model = estimator.fit(feature, lb.transform(target))
fit_model.save(filepath='./data/deep_learn.tf')